In [1]:
import numpy as np
import pandas as pd

In [56]:
np.seterr('raise')
df = pd.read_csv("../dmbi-18-ddi/train.csv")
df["is_linked"] = 1

def random_missing(df, size):
    max_node = max(df["node1"].max(), df["node2"].max())
    exist = set(df.T.apply(lambda x: (x['node1'],x['node2'])))
    chosen = set()
    while len(chosen) < size:
        zero_batch = np.random.randint(0, max_node, (2*size, 2))
        legal_idx = zero_batch[:,0] != zero_batch[:,1]
        zero_batch = zero_batch[legal_idx]
        zero_batch = set([tuple(row) for row in zero_batch]) - exist
        chosen.update(zero_batch)
    return np.hstack((np.array(list(chosen)[:size],dtype=np.int32),np.zeros((size,1),dtype=np.int32)))
    
def train_val_split(df):
    val_size = len(df)//1000
    val0 = random_missing(df, val_size*100)
    
    test_val1_idxs = np.random.choice(len(df), val_size, replace=False)
    val1 = df.iloc[test_val1_idxs]
    val_data = pd.DataFrame(np.vstack([val1,val0[val_size//10:]]), columns=["node1", "node2", "is_linked"])
    
    train_data = df.iloc[list(set(range(len(df)))-set(test_val1_idxs))]
    train_data = pd.DataFrame(np.vstack([train_data, val0[:val_size//10]]), columns=["node1", "node2", "is_linked"])
    return train_data, val_data

In [57]:
def get_batches(df, batch_size):
    indexes = np.random.permutation(len(df))
    for i in range(0, len(indexes), batch_size):
        parts = df.iloc[indexes[i:i+batch_size]]
        yield parts["node1"].values, parts["node2"].values, parts["is_linked"].values

In [58]:
train, val = train_val_split(df)
#train["is_linked"] = np.random.randint(0,2, len(train))

In [62]:
import tensorflow as tf
import tensorflow.contrib as contrib

n=1434
k=5
batch_size = 100
EPOCHS = 1000

tf.reset_default_graph()

regularizer = contrib.layers.l2_regularizer(1.0)

# Input should be [i1, ..., ib], [j1, ..., jb], [Gi1j1, ..., Gibjb]

# Placeholders
left_index = tf.placeholder(tf.int32, None, "left_index")
right_index = tf.placeholder(tf.int32, None, "right_index")
G_placehoder = tf.placeholder(tf.int32, None, "G")

# Variables
U = tf.get_variable("U", shape=(n, k), regularizer=regularizer)
b = tf.get_variable("b", shape=(n))

# Graph
inner = (
    tf.diag_part(tf.matmul(tf.gather(U, left_index), tf.transpose(tf.gather(U, right_index)))) 
    + tf.gather(b, left_index) + tf.gather(b, right_index)
)
output = tf.sigmoid(inner)

# Loss
reg_variables = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
error = tf.losses.mean_squared_error(G_placehoder, output) 
loss = error
if reg_variables:
    reg_term = contrib.layers.apply_regularization(regularizer, reg_variables)
    loss = loss + reg_term

init = tf.global_variables_initializer()
optimizer = tf.train.GradientDescentOptimizer(0.001).minimize(loss)

In [63]:
# Precision @ K
def average_precision_at_k(y, pred, k=100):
    #return average precision at k.
    #more examples: https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py
    #and: https://www.kaggle.com/c/avito-prohibited-content#evaluation
    #class_correct is a list with the binary correct label ordered by confidence level.
    ranked_pred = np.argsort(pred)[::-1][:k]
    assert k <= len(ranked_pred) and k > 0        
    score = 0.0
    hits = 0.0
    for i in range(k):
        if y[ranked_pred[i]]==1:
            hits += 1.0
        score += hits /(i+1.0)
    score /= k
    return score


In [ ]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(EPOCHS):
        train_loss = 0
        for (left, right, G) in get_batches(train, batch_size):
            t_loss, t_error, _ = sess.run([loss, error, optimizer], feed_dict={
                left_index : left,
                right_index : right,
                G_placehoder : G
            })
            train_loss += t_loss*batch_size
        
        print(train_loss / len(train))
        left_val, right_val, G_val = next(get_batches(val, len(val)))
        val_loss, val_error, val_pred = sess.run([loss, error, output], feed_dict={
                left_index : left_val,
                right_index : right_val,
                G_placehoder : G_val
            })
        print("Val loss: ", val_loss, "val_error:", val_error)
        print("Val Acc:", average_precision_at_k(G_val, val_pred))

1.43644344509
Val loss:  0.366688 val_error: 0.250202
Val Acc: 0.002343982386941037
0.311484633071
Val loss:  0.282613 val_error: 0.250351
Val Acc: 0.003688264742872629
0.271662020998
Val loss:  0.265355 val_error: 0.250499
Val Acc: 0.00428101998549236
0.260659071008
Val loss:  0.259155 val_error: 0.250647
Val Acc: 0.005075071046878826
0.255918140184
Val loss:  0.2563 val_error: 0.250795
Val Acc: 0.006493335669421709
0.253338591954
Val loss:  0.254794 val_error: 0.250943
Val Acc: 0.00708172179310195
0.251699430441
Val loss:  0.253935 val_error: 0.25109
Val Acc: 0.009762914014828594
0.250550123574
Val loss:  0.253425 val_error: 0.251238
Val Acc: 0.014121028117110343
0.249662066632
Val loss:  0.253119 val_error: 0.251386
Val Acc: 0.019968663148044737
0.248939394512
Val loss:  0.252941 val_error: 0.251534
Val Acc: 0.02577118441962094
0.24832636414
Val loss:  0.252847 val_error: 0.251682
Val Acc: 0.029487765097828432
0.24778029456
Val loss:  0.25281 val_error: 0.251829
Val Acc: 0.035370294

Val Acc: 0.1441253057213626
0.219381610956
Val loss:  0.264701 val_error: 0.264687
Val Acc: 0.14471410050146172
0.219089174163
Val loss:  0.264845 val_error: 0.264831
Val Acc: 0.14568432910326928
0.218796179705
Val loss:  0.264989 val_error: 0.264975
Val Acc: 0.1460179049950919
0.21851008666
Val loss:  0.265133 val_error: 0.265119
Val Acc: 0.1465459502751453
0.218227711384
Val loss:  0.265277 val_error: 0.265263
Val Acc: 0.14812616037301848
0.217946534893
Val loss:  0.26542 val_error: 0.265407
Val Acc: 0.14883999775582155
0.217666705373
Val loss:  0.265564 val_error: 0.265551
Val Acc: 0.14923114199179177
0.217375840647
Val loss:  0.265708 val_error: 0.265695
Val Acc: 0.1498857433511617
0.217101076027
Val loss:  0.265851 val_error: 0.265839
Val Acc: 0.15058773221510335
0.216814578927
Val loss:  0.265995 val_error: 0.265983
Val Acc: 0.15220776435055472
0.216526321104
Val loss:  0.266139 val_error: 0.266127
Val Acc: 0.15302553866981547
0.216244140389
Val loss:  0.266282 val_error: 0.26627